# Notebook for API debug

## Coinlore API -> [website](https://www.coinlore.com/cryptocurrency-data-api)

In [ ]:
import requests

class Client:

    def getglobal(self):
        '''
        Get global crypto statistics, including the total count of coins, overall market capitalization, BTC dominance, total trading volume, ATH market capitalization, and more
        '''
        self.r = requests.get('https://api.coinlore.com/api/global/')
        return self.r.json()[0]

    def getcoin(self, coin):
        '''
        coin: ID
        Retrieve tick data for specific coin, you should pass ID which will be given by /api/tickers/ endpoint, data includes details such as name, ID, symbol, price, price change, market cap, volume, and supply for each ticker.
        '''
        self.r = requests.get('https://api.coinlore.com/api/ticker/?id=' + str(coin))
        return self.r.json()[0]

    def getcoins(self, start='0', limit='100'):
        '''
        start: str numeric - start index of coins sorted by market cap
        limit: str numeric - limit number of coins to return
        MAXIMUM LIMIT: 100 per request
        
        Retrieve tick data for multiple crypto coins, sorted by market cap. The data includes details such as name, ID, symbol, price, price change, market cap, volume, and supply for each ticker.
        '''
        self.r = requests.get('https://api.coinlore.com/api/tickers/?start=' + str(start) + '&limit=' + str(limit))
        return self.r.json()

    def getmarkets(self, coin):
        self.r = requests.get('https://api.coinlore.com/api/coin/markets/?id=' + str(coin))
        return self.r.json()

    def getsocial(self, coin):
        self.r = requests.get('https://api.coinlore.com/api/coin/social_stats/?id=' + str(coin))
        return self.r.json()

In [ ]:
from coinlore.client import Client

client = Client()

#Get global market info
print(client.getglobal())

#Get Bitcoin Info (Bitcoin)
print(client.getcoin(90))

#Get coins from 0 to 100
print(client.getcoins("0", "100"))

#Get coin markets (Bitcoin)
print(client.getmarkets(90))

#Get social stats (Ethereum)
print(client.getsocial(80))


{'coins_count': 14542, 'active_markets': 36086, 'total_mcap': 3452793498816.4067, 'total_volume': 120001669176.84729, 'btc_d': '62.77', 'eth_d': '9.17', 'mcap_change': '0.83', 'volume_change': '-2.08', 'avg_change_percent': '0.19', 'volume_ath': 344187126292428700, 'mcap_ath': 33242498693028.46}
{'id': '90', 'symbol': 'BTC', 'name': 'Bitcoin', 'nameid': 'bitcoin', 'rank': 1, 'price_usd': '109079.33', 'percent_change_24h': '0.28', 'percent_change_1h': '0.22', 'percent_change_7d': '3.29', 'price_btc': '1.00', 'market_cap_usd': '2167032982582.20', 'volume24': 24249258801.352173, 'volume24a': 26400008140.4453, 'csupply': '19866577.00', 'tsupply': '19866577', 'msupply': '21000000'}
{'data': [{'id': '90', 'symbol': 'BTC', 'name': 'Bitcoin', 'nameid': 'bitcoin', 'rank': 1, 'price_usd': '109079.33', 'percent_change_24h': '0.28', 'percent_change_1h': '0.22', 'percent_change_7d': '3.29', 'price_btc': '1.00', 'market_cap_usd': '2167032982582.20', 'volume24': 24249258801.352173, 'volume24a': 26400

# Alpha Vantage <- only 1 hours interval. 1min for premium

In [ ]:
from alpha_vantage.cryptocurrencies import CryptoCurrencies
import matplotlib.pyplot as plt

API_KEY = ''
CRYPTO_SYMBOL = 'BTC'
MARKET = 'USD'  
INTERVAL = '1min'  
OUTPUT_SIZE = 'compact' 

cc = CryptoCurrencies(
    key=API_KEY,
    output_format='pandas',
    indexing_type='date'     
)

data, meta_data = cc.get_digital_currency_daily(
    symbol=CRYPTO_SYMBOL,
    market=MARKET,
    # interval=INTERVAL,
    # outputsize=OUTPUT_SIZE
)


print("Метаданные:")
print(meta_data)
print("\nПоследние 5 записей:")
print(data.head())



Метаданные:
{'1. Information': 'Daily Prices and Volumes for Digital Currency', '2. Digital Currency Code': 'BTC', '3. Digital Currency Name': 'Bitcoin', '4. Market Code': 'USD', '5. Market Name': 'United States Dollar', '6. Last Refreshed': '2025-07-09 00:00:00', '7. Time Zone': 'UTC'}

Последние 5 записей:
              1. open    2. high     3. low   4. close    5. volume
date                                                               
2025-07-09  108953.58  108987.73  108806.75  108849.69    35.528228
2025-07-08  108271.49  109255.99  107438.33  108958.04  3785.390742
2025-07-07  109217.98  109741.64  107507.00  108269.84  4455.083530
2025-07-06  108246.66  109736.64  107837.70  109217.98  1651.181202
2025-07-05  108028.60  108454.19  107783.10  108246.65  1513.154286


KeyError: '4b. close (USD)'

# Binance API <- храни тебя господь родной

In [20]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
import json
import os

class BinanceDataCollector:
    def __init__(self):
        self.base_url = "https://api.binance.com/api/v3"
        self.session = requests.Session()
        
    def get_klines(self, symbol, interval, start_time, end_time, limit=1000):
        """
        Получает данные свечей с Binance API
        
        Args:
            symbol: торговая пара (например, 'BTCUSDT')
            interval: интервал ('1m', '5m', '1h', '1d')
            start_time: начальное время (timestamp в миллисекундах)
            end_time: конечное время (timestamp в миллисекундах)
            limit: количество свечей за запрос (макс 1000)
        """
        url = f"{self.base_url}/klines"
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': limit
        }
        
        try:
            response = self.session.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при запросе данных: {e}")
            return None
    
    def collect_historical_data(self, symbol, interval, days_back=730):
        """
        Собирает исторические данные за указанный период
        
        Args:
            symbol: торговая пара
            interval: интервал
            days_back: количество дней назад (по умолчанию 730 = 2 года)
        """
        print(f"Начинаем сбор данных для {symbol} с интервалом {interval}")
        
        end_time = datetime.now()
        start_time = end_time - timedelta(days=days_back)
        
        start_timestamp = int(start_time.timestamp() * 1000)
        end_timestamp = int(end_time.timestamp() * 1000)
        
        all_data = []
        current_start = start_timestamp
        
        interval_ms = self._get_interval_ms(interval)
        
        while current_start < end_timestamp:
            current_end = min(current_start + (1000 * interval_ms), end_timestamp)
            
            print(f"Загружаем данные с {datetime.fromtimestamp(current_start/1000)} по {datetime.fromtimestamp(current_end/1000)}")
            
            klines = self.get_klines(symbol, interval, current_start, current_end)
            
            if klines:
                all_data.extend(klines)
                print(f"Получено {len(klines)} свечей")
            else:
                print("Ошибка при получении данных")
                break
            
            current_start = current_end + interval_ms
            
            time.sleep(0.1)
        
        return all_data
    
    def _get_interval_ms(self, interval):
        """Конвертирует интервал в миллисекунды"""
        intervals = {
            '1m': 60 * 1000,
            '3m': 3 * 60 * 1000,
            '5m': 5 * 60 * 1000,
            '15m': 15 * 60 * 1000,
            '30m': 30 * 60 * 1000,
            '1h': 60 * 60 * 1000,
            '2h': 2 * 60 * 60 * 1000,
            '4h': 4 * 60 * 60 * 1000,
            '6h': 6 * 60 * 60 * 1000,
            '8h': 8 * 60 * 60 * 1000,
            '12h': 12 * 60 * 60 * 1000,
            '1d': 24 * 60 * 60 * 1000,
            '3d': 3 * 24 * 60 * 60 * 1000,
            '1w': 7 * 24 * 60 * 60 * 1000,
            '1M': 30 * 24 * 60 * 60 * 1000
        }
        return intervals.get(interval, 60 * 1000)
    
    def process_data_to_dataframe(self, raw_data):
        """
        Преобразует сырые данные в pandas DataFrame
        """
        if not raw_data:
            return pd.DataFrame()
        
        df = pd.DataFrame(raw_data, columns=[
            'open_time', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'number_of_trades',
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
        ])
        
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
        
        price_volume_columns = ['open', 'high', 'low', 'close', 'volume', 
                               'quote_asset_volume', 'taker_buy_base_asset_volume', 
                               'taker_buy_quote_asset_volume']
        
        for col in price_volume_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df['number_of_trades'] = pd.to_numeric(df['number_of_trades'], errors='coerce')
        
        df = df.drop('ignore', axis=1)
        
        df = df.sort_values('open_time').reset_index(drop=True)
        
        return df
    
    def save_data(self, df, filename):
        """Сохраняет данные в CSV файл"""
        os.makedirs('data', exist_ok=True)
        filepath = f'data/{filename}'
        df.to_csv(filepath, index=False)
        print(f"Данные сохранены в {filepath}")
    
    def load_data(self, filename):
        """Загружает данные из CSV файла"""
        filepath = f'data/{filename}'
        if os.path.exists(filepath):
            df = pd.read_csv(filepath)
            df['open_time'] = pd.to_datetime(df['open_time'])
            df['close_time'] = pd.to_datetime(df['close_time'])
            return df
        else:
            print(f"Файл {filepath} не найден")
            return None

def main():
    collector = BinanceDataCollector()
    
    symbol = 'BTCUSDT'  
    interval = '5m'     
    days_back = 365     # Days back 
    
    print("Начинаем сбор данных...")
    raw_data = collector.collect_historical_data(symbol, interval, days_back)
    
    if raw_data:
        print(f"Собрано {len(raw_data)} свечей")
        
        df = collector.process_data_to_dataframe(raw_data)
        
        print("\nИнформация о данных:")
        print(f"Период: с {df['open_time'].min()} по {df['open_time'].max()}")
        print(f"Количество записей: {len(df)}")
        print(f"Размер данных: {df.shape}")
        
        print("\nПервые 5 записей:")
        print(df.head())
        
        filename = f'{symbol}_{interval}_{days_back}days.csv'
        collector.save_data(df, filename)
        
        print("\nБазовая статистика по ценам:")
        print(df[['open', 'high', 'low', 'close', 'volume']].describe())
    
    else:
        print("Не удалось собрать данные")

def collect_multiple_coins():
    collector = BinanceDataCollector()
    
    symbols = ['BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'DOTUSDT']
    interval = '5m'  
    days_back = 365  
    
    for symbol in symbols:
        print(f"\n{'='*50}")
        print(f"Собираем данные для {symbol}")
        print(f"{'='*50}")
        
        raw_data = collector.collect_historical_data(symbol, interval, days_back)
        
        if raw_data:
            df = collector.process_data_to_dataframe(raw_data)
            filename = f'{symbol}_{interval}_{days_back}days.csv'
            collector.save_data(df, filename)
            
            print(f"✅ Данные для {symbol} сохранены: {len(df)} записей")
        else:
            print(f"❌ Ошибка при сборе данных для {symbol}")
        
        time.sleep(1)

if __name__ == "__main__":
    main()
    
    # Раскомментируйте для сбора данных по нескольким монетам
    # collect_multiple_coins()

Начинаем сбор данных...
Начинаем сбор данных для BTCUSDT с интервалом 5m
Загружаем данные с 2024-07-09 15:38:57.203000 по 2024-07-13 02:58:57.203000
Получено 1000 свечей
Загружаем данные с 2024-07-13 03:03:57.203000 по 2024-07-16 14:23:57.203000
Получено 1000 свечей
Загружаем данные с 2024-07-16 14:28:57.203000 по 2024-07-20 01:48:57.203000
Получено 1000 свечей
Загружаем данные с 2024-07-20 01:53:57.203000 по 2024-07-23 13:13:57.203000
Получено 1000 свечей
Загружаем данные с 2024-07-23 13:18:57.203000 по 2024-07-27 00:38:57.203000
Получено 1000 свечей
Загружаем данные с 2024-07-27 00:43:57.203000 по 2024-07-30 12:03:57.203000
Получено 1000 свечей
Загружаем данные с 2024-07-30 12:08:57.203000 по 2024-08-02 23:28:57.203000
Получено 1000 свечей
Загружаем данные с 2024-08-02 23:33:57.203000 по 2024-08-06 10:53:57.203000
Получено 1000 свечей
Загружаем данные с 2024-08-06 10:58:57.203000 по 2024-08-09 22:18:57.203000
Получено 1000 свечей
Загружаем данные с 2024-08-09 22:23:57.203000 по 2024-